# parse_logfile

This dict is the result of serialising (then cleaning) a logfile which is dumped when the assertion on line ~700 fails:
```
assert target.name not in self.converter._function_definitions, target.name
```

Look at this dict, parse how the three relevant objects are filled (really two are objects, one is a call history)

In [1]:
import json


from typed_python import SerializationContext

In [2]:
SERIALIZED_INPUT = '/home/wgrant/Dev/typed_python/demos/python_s_slow_u_slice_u_3/logfile_20221122.bytes'
# SERIALIZED_INPUT = '/home/wgrant/Dev/typed_python/logfile.bytes'

In [3]:
!ls -l {SERIALIZED_INPUT}

-rw-r--r-- 1 wgrant wgrant 64986 Nov 22 18:46 /home/wgrant/Dev/typed_python/demos/python_s_slow_u_slice_u_3/logfile_20221122.bytes


In [4]:
with open(SERIALIZED_INPUT, 'rb') as flines:
    input_data = SerializationContext().deserialize(flines.read())

In [5]:
# PATH_TO_INPUT = '/home/wgrant/Dev/typed_python/demos/failure_dict.json'
# with open(PATH_TO_INPUT) as flines:
#     input_data = json.load(flines)

In [6]:
for key, value in input_data.items():
    print(key, len(value))

history 4
end_state 3
compiler_cache 4
bug_test 0


In [7]:
import pandas as pd 

In [8]:
pd.set_option('display.max_rows', 2000)

## History

In [9]:
def parse_dict_to_dataframe(input_dict: dict) -> pd.DataFrame:
    input_data_squared = []
    for key, value in input_dict.items():
        for x in value:
            try:
                timestamp, thread, data = x
            except TypeError:
                timestamp, thread, data = x, None
            input_data_squared.append((pd.to_datetime(timestamp, unit='s'), thread, key, data))
    input_df = pd.DataFrame(input_data_squared, columns=['timestamp', 'thread', 'key', 'data']).sort_values(by='timestamp').reset_index(drop=True)
    
    # extract the callTarget data
    callTarget_data = pd.json_normalize(input_df.query('key=="namedCallTargetToLLVM"').data).set_index(input_df.query('key=="namedCallTargetToLLVM"').index)
    callTarget_data.columns = ['name', 'namedCallTargetToLLVM.external']
    parsed_df = pd.merge(input_df, callTarget_data, how='left', left_index=True, right_index=True)
    parsed_df.loc[parsed_df.key=="namedCallTargetToLLVM", 'data'] = None
    # set the name field as data for function_definitions
    parsed_df.loc[parsed_df.key=="function_definitions", 'name'] = parsed_df.loc[parsed_df.key=="function_definitions", :].data  
    parsed_df.loc[parsed_df.key=="external_function_references", 'name'] = parsed_df.loc[parsed_df.key=="external_function_references", :].data  
    # blow up the externallyDefinedFunctionTypes
    parsed_df = parsed_df.explode('data')
    parsed_df.loc[parsed_df.key=="externallyDefinedFunctionTypes", 'name'] = parsed_df.loc[parsed_df.key=="externallyDefinedFunctionTypes", :].data
    # grab entryPoint names
    parsed_df.loc[parsed_df.key=="entryPoint", 'name'] = input_df.loc[input_df.key=='entryPoint', 'data'].apply(lambda x: x['name'])
    
    parsed_df = parsed_df.drop(columns=['data']).reset_index(drop=True)
    return parsed_df, input_df

In [10]:
df, input_df = parse_dict_to_dataframe(input_data['history'])

In [11]:
import numpy as np

In [12]:
def bold(row):
    if not pd.isnull(row['name']) and 'decref_str' in row['name']:
        return ['background-color: #1b9e77'] * len(row)
    elif row['name'] == 'CLEAR':
        return ['background-color: #d95f02'] * len(row)
        
    else:
        return [None] * len(row)

In [13]:
# df.style.apply(bold, axis=1)

In [14]:
# df[df.key == 'external_function_references']

In [15]:
# df[(df.name.notnull() & df.name.str.contains('decref_str')) | (df.key == 'external_function_references')]

In [16]:
repeated_names = df[df.key.isin(['function_definitions', 'externallyDefinedFunctionTypes'])].name.value_counts()

In [17]:
repeated_names = repeated_names[repeated_names > 1].index

In [18]:
repeated_names

Index([], dtype='object')

In [19]:
# df[df.name.isin(repeated_names)]

## End State

In [20]:
def parse_dict_to_end_state_dataframe(input_dict): 
    protodf = []
    for key, value in input_dict['end_state'].items():
        if key == 'namedCallTargetToLLVM':
            continue
        timestamp, thread, function_names = value
        for name in function_names:
            protodf.append((pd.to_datetime(timestamp, unit='s'), thread, key, name))
    end_state = pd.DataFrame(protodf, columns=['timestamp', 'thread', 'key', 'name']).sort_values(by=['key', 'name'])
    return end_state

In [21]:
# parse_dict_to_end_state_dataframe(input_data).style.apply(bold,axis=1) 

In [22]:
input_data['end_state']['namedCallTargetToLLVM']

[1669160808.2517753,
 2005,
 {'name': 'runtime.type_convert(NoneType -> OneOf(None, int), conversionLevel=4).35c4c96d5009d1f818454d90c070b56da027a651',
  'external': False}]

## Compiler Stuff

In [23]:
for key, value in input_data.items():
    print(key, len(value))

history 4
end_state 3
compiler_cache 4
bug_test 0


In [24]:
def parse_dict_to_compiler_cache_dataframe(input_dict): 
    protodf = []
    for key, value in input_dict['compiler_cache'].items():
        for timestamp, thread, function_name in value:
            protodf.append((pd.to_datetime(timestamp, unit='s'), thread, key, function_name))
    compiler_cache = pd.DataFrame(protodf, columns=['timestamp', 'thread', 'key', 'name']).sort_values(by='timestamp')
    return compiler_cache

In [25]:
compiler_df =parse_dict_to_compiler_cache_dataframe(input_data)

In [26]:
compiler_df[compiler_df.name.str.contains('decref_str')]

,timestamp,thread,key,name
112,2022-11-22 23:46:47.666805248,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
332,2022-11-22 23:46:47.836373504,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
627,2022-11-22 23:46:47.923132416,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...


In [27]:
# compiler_df.style.apply(bold,axis=1)

## Bug Test - 


In [28]:
def parse_dict_to_bug_test_dataframe(input_dict): 
    protodf = []
    for timestamp, thread, value in input_dict['bug_test']:
        for key, vals in value.items():
            for function_name in vals:
                protodf.append((pd.to_datetime(timestamp, unit='s'), thread, key, function_name))
    bug_test = pd.DataFrame(protodf, columns=['timestamp', 'thread', 'key', 'name']).sort_values(by=['timestamp', 'key']).reset_index(drop=True)
    return bug_test

In [29]:
# bug_test_df = parse_dict_to_bug_test_dataframe(input_data)

In [30]:
# bug_test_df[bug_test_df.name.str.contains('decref_str')]

In [31]:
# bug_test_df.style.apply(bold, axis=1)

In [32]:
# rows = []
# for index, row in bug_test_df.query('key=="markExternal"').iterrows():
#     name = row['name']
#     preceding_rows = bug_test_df.iloc[:index].query('name==@name and key=="definedNames"')
#     if len(preceding_rows):   
#         # rows.append(pd.concat([preceding_rows, row]))
#         rows.append(preceding_rows.append(row))

In [33]:
# defined_then_loaded = pd.concat(rows)

In [34]:
# defined_then_loaded

In [35]:
# defined_then_loaded_names = defined_then_loaded['name'].unique()

In [36]:
# print("\n".join(sorted(defined_then_loaded['name'].unique())))

In [37]:
# print("\n".join(sorted(repeated_names)))

In [38]:
# defined_then_loaded

In [39]:
# merged[merged.name.isin(defined_then_loaded['name'].unique())].style

In [40]:
repeated_names

Index([], dtype='object')

In [41]:
# bug_test_df

## Combo

In [42]:
history_df = df.drop(columns='namedCallTargetToLLVM.external')

In [43]:
history_df.shape

(579, 4)

In [44]:
compiler_df.shape

(1030, 4)

In [45]:
merged = pd.concat([history_df, compiler_df]).sort_values(by=['timestamp', 'key', 'name'])

In [46]:
# merged

In [47]:
merged[merged.name.str.contains('decref_str')]

,timestamp,thread,key,name
112,2022-11-22 23:46:47.666805248,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
332,2022-11-22 23:46:47.836373504,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
627,2022-11-22 23:46:47.923132416,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
508,2022-11-22 23:46:48.196358400,2005,namedCallTargetToLLVM,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
509,2022-11-22 23:46:48.196428032,2005,external_function_references,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...


In [48]:
# merged[merged.key.isin(['namedCallTargetToLLVM', 'external_function_references'])]

In [49]:
# df[df['namedCallTargetToLLVM.external'] == False]

In [50]:
# merged

In [51]:
# check end_state to debug how on earth NCT2LLVM got called thrice

In [52]:
# df.iloc[-100:]

In [53]:
# end_state_df = parse_dict_to_end_state_dataframe(input_data)

In [54]:
# end_state_df[end_state_df.name.str.contains('checkSetSizeAndThrow')]

In [55]:
# end_state_df.query("key=='function_definitions'")

So what happened:
-  defineLinkName called with RDS - added to `_allDefinedNames_` and then loadFromCompilerCache hit. symbol found in the cache but either callTargetsAndTypes was None, or newNativeFunctionTypes is empty.
- then RDS added to function_definitions.
- call NCT2LLVM a bunch of times. 
- then load RDS again, as a result of loadFromCompilerCaching a different function **which function**. RDS gets in EDFTs, ADS, ACS
- then three more NCT2LLVM hits, and only the third breaks?

In [56]:
# merged.loc[1048]

In [57]:
# merged.index.get_loc(1048)

In [58]:
# merged[600:792]

In [59]:
# merged[791:1000]

For each function, get the first occurences of namedCallTargetToLLVM, and all external_function_references

In [60]:
# repeated_names

In [61]:
# external_function_refs = merged.query("key=='external_function_references'")
# namedCallTargetToLLVM_firstcall = merged.query("key=='namedCallTargetToLLVM'").groupby('name').first().reset_index()
# pd.concat([namedCallTargetToLLVM_firstcall, external_function_refs]).sort_values(by='timestamp')[['timestamp', 'thread', 'key', 'name']].style

In [62]:
# defined_then_loaded

In [63]:
# history_df.query("key=='function_definitions'")

In [64]:
# merged[merged.name.str.contains('decref_str')]

In [65]:
# merged.query("key=='external_function_references'")

In [66]:
# repeated_names

In [67]:
# defined_then_loaded_names

In [68]:
# external_referenced_names = merged.query("key=='external_function_references'").name.unique()

In [69]:
# set(defined_then_loaded_names) - set(repeated_names )

In [70]:
merged[merged.name.str.contains('decref_str')]

,timestamp,thread,key,name
112,2022-11-22 23:46:47.666805248,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
332,2022-11-22 23:46:47.836373504,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
627,2022-11-22 23:46:47.923132416,2005,load_results,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
508,2022-11-22 23:46:48.196358400,2005,namedCallTargetToLLVM,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...
509,2022-11-22 23:46:48.196428032,2005,external_function_references,runtime.decref_str.6ff674b5be11a1473ce627fb8bd...


In [71]:
merged.reset_index(drop=True).style.apply(bold,axis=1)

,timestamp,thread,key,name
0,2022-11-22 23:46:44.558111744,2001,function_definitions,CLEAR
1,2022-11-22 23:46:44.558223872,2001,entryPoint,getNonblocking
2,2022-11-22 23:46:44.558497280,2001,entryPoint,get
3,2022-11-22 23:46:44.558671104,2001,entryPoint,getMany
4,2022-11-22 23:46:44.558844672,2001,entryPoint,put
5,2022-11-22 23:46:44.558963200,2001,entryPoint,putMany
6,2022-11-22 23:46:44.559899648,2001,entryPoint,peek
7,2022-11-22 23:46:44.560040704,2001,entryPoint,__len__
8,2022-11-22 23:46:44.560932096,2001,entryPoint,workExecutor
9,2022-11-22 23:46:44.561108736,2001,entryPoint,pmap
